In [1]:
from flask import Flask, request
from joblib import load

In [4]:
pipe = load('ml_classifier.joblib')

In [5]:
pipe

Pipeline(steps=[('encoder',
                 CountEncoder(cols=['Make', 'Year', 'Model'],
                              combine_min_nan_groups=True)),
                ('scaler', StandardScaler()),
                ('meka',
                 Meka(java_command='C:/Program Files '
                                   '(x86)/Java/jre1.8.0_281/bin/java -Xmx1024m',
                      meka_classifier='meka.classifiers.multilabel.meta.BaggingML '
                                      '-I 10 -P 100',
                      meka_classpath='C:\\Users\\Nico\\scikit_ml_learn_data\\meka\\meka-release-1.9.2\\lib\\'))])

In [36]:
app = Flask(__name__)

PORT=5000

@app.route("/")
def hello():
    return "Hello, World!"

@app.route("/recommend", methods=["POST"])
def score_inputs():
    serv_cats = ['Accessories (Gauges, Power Accessories, Vision & Air Bag)', 'Brakes',
       'Change Oil', 'Complete Vehicle Inspection', 'Cooling System',
       'Drive Train (Final Drive, Transaxle, Differential & Transfer Case)',
       'Driveability (Fuel, Ignition & Emission Systems)',
       'Electrical (Belts, Lighting, Starting & Charging Systems)', 'Engine',
       'Engine Oil / Oil Filter', 'Fuel Filter / Air Filter',
       'HVAC (Heating, Ventilation & Air Conditioning)',
       'Major Preventive Maintenance Service',
       'Minor Preventive Maintenance Service', 'Tires & Wheels',
       'Transmission & Clutch']
    
    content = request.json
    samp = content["values"]
    input_series = pd.Series(samp, index=['Make', 'Year', 'Model', 'Mileage In'])
    input_to_df = input_series.values.reshape(1,4)
    input_df = pd.DataFrame(input_to_df, columns=['Make', 'Year', 'Model', 'Mileage In'])
    
    result = pipe.predict(input_df).todense().tolist()
    binar = np.array(result).reshape(-1)
    result_df = pd.DataFrame({"binar":binar, "serv_cats":serv_cats})
    recomms = result_df[ result_df['binar']==1 ]['serv_cats'].values.tolist()
    
    return {"result": recomms}

if __name__ == '__main__':
  app.run(port=PORT)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2021 19:07:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2021 19:07:44] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2021 19:07:56] "POST /recommend HTTP/1.1" 200 -
